## Import Dependencies

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

## Run the webscraper

In [ ]:
# set up the driver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome("/Users/macdkw/Downloads/chromedriver")

# Navigate to the page
def home():
    driver.get("https://www.texasmusicforms.com/csrrptUILpublic.asp")

# get results button function
def click_results_button():
    get_results_button = driver.find_element(by=By.CLASS_NAME, value="btn-success")
    get_results_button.click()


# "Search again" button function
def click_search_button():
    search_again_button = driver.find_element(by=By.NAME, value="chan")
    search_again_button.click()
    driver.implicitly_wait(10)

# find the drop down menus and fill them out
def form_completer(year, region, contest_item):
    year_menu = driver.find_element(by=By.NAME, value="yr")
    region_menu = driver.find_element(by=By.NAME, value="reg")
    event_menu = driver.find_element(by=By.NAME, value="ev")
    year_menu.send_keys(year)
    region_menu.send_keys(f"Region {region}")
    event_menu.send_keys("Band")

    # find the hidden drop down menu, and collect its contents
    contest_menu = driver.find_element(by=By.NAME, value="cn")
    contest_menu_contents = contest_menu.find_elements(by=By.TAG_NAME, value="option")
    contest_menu_length = len(contest_menu_contents)
    contest_menu.send_keys(contest_menu_contents[contest_item].text)


# scrape the data
def scraper_loop():
    # make sure that were seeing all results, not just the top 20
    show_all_button = driver.find_element(by=By.NAME, value="DataTables_Table_0_length")
    show_all_button.send_keys("All")
    # get the table
    table = driver.find_elements(by=By.TAG_NAME, value="tbody")[1]
    table_rows = table.find_elements(by=By.TAG_NAME, value="tr")
    # loop through table contents
    for row in table_rows:
        cells = row.find_elements(by=By.TAG_NAME, value="td")
        column_content = []
        for cell in cells:
            cell_content_full = cell.text
            cell_content = cell_content_full.split("\n")
            for i in cell_content:
                column_content.append(i)
            #column_content.append(cell_content)
        row_content.append(column_content)

# convert to dataframe
def convert_to_dataframe():
    df = pd.DataFrame(row_content)
    if len(df.columns) == 21:
        df.columns = columns_21
    elif len(df.columns) == 22:
        df.columns = columns_22
    # export to csv
    df.to_csv(f"full_run/texas_music_forms_{year}_{region}_{contest_item}_band.csv")

columns_21 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3']

columns_22 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'oops']

row_content = []

# Define year and region variables
year = 2017
region = 0

for y in range(2019, 2023):
    for r in range(0, 33):
        try:
            contest_item = 1
            region = region + 1
            for i in range(1, 15):
                try:
                    home()
                    form_completer(year, region, contest_item)
                    click_results_button()
                    scraper_loop()
                    convert_to_dataframe()
                    contest_item = contest_item + 1
                    row_content = []
                except:
                    print(f"{year}, {region}, {contest_item}")
                    break
        except:
            break
    year = year + 1
    region = 0

In [3]:
### Define column headers for csv export
columns_21 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3']
columns_22 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'oops']

## Group the CSVs together, and fix rows with Accompanist error

In [4]:
# setting the path for joining multiple files
files = os.path.join("/Users/macdkw/Class/uil_results/full_run/", "*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df = pd.DataFrame(df)

# drop unnamed column
df.drop(columns=['Unnamed: 0'], inplace=True)

# select rows with acc column 
oops_df = df[df['oops'].notnull()]

# delete the acc columns and fix column names
oops_fix_df = oops_df.drop(columns=['Conference'])
oops_fix_df.columns = columns_21

# drop acc rows from df
df = df.loc[df['oops'].isnull() == True]

# add fixed df to df
df = pd.concat([df, oops_fix_df], ignore_index=True)

# drop oops column
df.drop(columns=['oops'], inplace=True)



## Clean the data further

In [5]:
# drop duplicate rows
df_clean = df
df_clean.drop_duplicates(keep='first', inplace=True)
df_clean.sort_values(by=['Classification', 'Year'], inplace=True)
df_clean



Event                       School               TEA  \
14433   100-Concert Band         Progreso High School              TEA:   
17136   100-Concert Band                     Bangs HS              TEA:   
24128   100-Concert Band               Woodville High              TEA:   
44305   100-Concert Band         New Deal High School              TEA:   
8245    100-Concert Band             Alicia R. Chacon              TEA:   
...                  ...                          ...               ...   
45107   100-Concert Band    Warren Junior High School              TEA:   
59063   100-Concert Band                   McCamey MS              TEA:   
10009   100-Concert Band   Lake Air Montessori Magnet       TEA: 161914   
23172   100-Concert Band                    Alpine MS   TEA: 022 901041   
52318   100-Concert Band                     Eastland              TEA:   

             City                                      Directors Conference  \
14433    Progreso                                    James Yates        AAA   
17136       Bangs                 Tommy Perkins / Charla Bretzke         AA   
24128   Woodville                                      Lou Ellis         AA   
44305    New Deal                        James Nance / Kay Nance          A   
8245      El Paso   Jose Barriga / Salvador Melendez, Louie Nava          C   
...           ...                                            ...        ...   
45107      Warren                  Carl Hooper / Russell Hopkins        AAA   
59063     McCamey                  Shawna Torres / German Torres          C   
10009        Waco           Todd Modgling / Stephanie Fernihough          C   
23172      Alpine                                    Marina Azar          C   
52318    Eastland    Stephen Cox / Terry Everts, Dillon Phillips         AA   

      Classification  Year      ID Stage Judge 1  ... Stage Judge 3  \
14433                 2009   48134             3  ...             2   
17136                 2009   43637             1  ...             1   
24128                 2009   46343             2  ...             2   
44305                 2009   45190             2  ...             1   
8245                  2010   59515             1  ...             1   
...              ...   ...     ...           ...  ...           ...   
45107      Varsity C  2016  116407             4  ...             4   
59063      Varsity C  2018  129652             2  ...             2   
10009      Varsity C  2022  175313             2  ...             1   
23172      Varsity C  2022  172064             1  ...             1   
52318      Varsity C  2014   91726             2  ...             2   

      Stage Final SR Judge 1 SR Judge 2 SR Judge 3 SR Final Award  \
14433           2          2          2          2        2         
17136           1          1          1          1        1     A   
24128           2          1          1          1        1     1   
44305           2          1          1          1        1     1   
8245            1          2          1          1        1     C   
...           ...        ...        ...        ...      ...   ...   
45107           4          2          1          1        1     2   
59063           2          1          2          1        1     2   
10009           2          1          2          1        1     2   
23172           1          1          1          1        1     C   
52318           2          1          2          1        1     2   

                                           Selection 1  \
14433                    Portrait of a Clown (Ticheli)   
17136                        Monterey March (LaPlante)   
24128   Under The Double Eagle - March (Wagner/Glover)   
44305                 La Fiesta Brava (John Edmondson)   
8245                       Lexington March (Edmondson)   
...                                                ...   
45107             Swashbuckler's march (Mark Williams)   
59063                  

In [6]:
# select rows where selection 3 is null
df_clean_selection_3_null = df_clean[df_clean['Selection 3'].isnull()]
df_clean_selection_3_null

Event                  School    TEA          City  \
13328   100-Concert Band                       0   TEA:                 
13230   100-Concert Band                       0   TEA:                 
9117    100-Concert Band                       0   TEA:                 
26026   100-Concert Band                       0   TEA:                 
37724   100-Concert Band                       0   TEA:                 
...                  ...                     ...    ...           ...   
43798   100-Concert Band     Del Rio High School   TEA:       Del Rio   
52316   100-Concert Band             Alvarado MS   TEA:      Alvarado   
52315   100-Concert Band             Alvarado MS   TEA:      Alvarado   
25167   100-Concert Band   Stinson Middle School   TEA:   San Antonio   
20496   100-Concert Band     Del Rio High School   TEA:       Del Rio   

                          Directors        Conference Classification    Year  \
13328                         cathy          Varsity            2005   11109   
13230               Mandi McCasland          Varsity            2005   11197   
9117                     lflisowski          Varsity            2005   12205   
26026                     sbostwick      Non-Varsity            2005   12811   
37724                        cjcain          Varsity            2005   13014   
...                             ...               ...            ...     ...   
43798                  Daniel White  Sub Non-Varsity            2014   92973   
52316       Kelli Bahner / Joe Gunn      Non-Varsity            2014   92415   
52315       Kelli Bahner / Joe Gunn          Varsity            2014   92418   
25167   Kevin Leman / Alex Melendez      Non-Varsity            2014   95724   
20496                  Daniel White      Non-Varsity            2015  104122   

      ID Stage Judge 1  ... Stage Judge 3 Stage Final SR Judge 1 SR Judge 2  \
13328  1             1  ...             1           1          1          1   
13230                   ...                                                   
9117   1             1  ...             1           2          2          2   
26026  4             4  ...             4           4          4          4   
37724  2             2  ...             2           1          1          1   
...   ..           ...  ...           ...         ...        ...        ...   
43798  1             1  ...             1           1          2          1   
52316  1             1  ...             1           2          2          2   
52315  1             1  ...             1           1          1          1   
25167  1             1  ...             1           1          1          1   
20496  1             2  ...             2           1          1          1   

      SR Judge 3 SR Final                                        Award  \
13328          1      TRC                       Escapada (David Moore)   
13230                                         Valdres (Hannsen/Curnow)   
9117           2      RM1       War March of the Tartars (King, K. L.)   
26026          4        -       Blue Ribbon March (Feldstein-O'Reilly)   
37724          1      RM1                     Newcastle March (Vinson)   
...          ...      ...                                          ...   
43798          1        C                     Broken Bow  (Strommen/ )   
52316          2        2                        Ghost Ship  (Story/ )   
52315          1        C             Fields of Clover  (Standridge/ )   
25167          1        D                  Forward March (Paul Murtha)   
20496          1        2   El Capitan (J.P.Sousa/K.Brion, L.Schissel)   

                                  Selection 1  \
13328           Whitewater Festival (Shaffer)   
13230      Down Longford Way (Grainger/Osmon)   
9117     Shepherd's Hey (Grainger/Rogers/Mas)   
26026              Sea Song Trilogy (McGinty)   
37724     The Battle Pavane (Susato/Margolis)   
...                                       ...   


In [ ]:
df_clean_selection_3_null['Selection 3'] = df_clean_selection_3_null['Selection 2']
df_clean_selection_3_null['Selection 2'] = df_clean_selection_3_null['Selection 1']
df_clean_selection_3_null['Selection 1'] = df_clean_selection_3_null['Award']
df_clean_selection_3_null['Award'] = df_clean_selection_3_null['SR Final']
df_clean_selection_3_null['SR Final'] = df_clean_selection_3_null['SR Judge 3']
df_clean_selection_3_null['SR Judge 3'] = df_clean_selection_3_null['SR Judge 2']
df_clean_selection_3_null['SR Judge 2'] = df_clean_selection_3_null['SR Judge 1']
df_clean_selection_3_null['SR Judge 1'] = df_clean_selection_3_null['Stage Final']
df_clean_selection_3_null['Stage Final'] = df_clean_selection_3_null['Stage Judge 3']
df_clean_selection_3_null['Stage Judge 3'] = df_clean_selection_3_null['Stage Judge 2']
df_clean_selection_3_null['Stage Judge 2'] = df_clean_selection_3_null['Stage Judge 1']
df_clean_selection_3_null['Stage Judge 1'] = df_clean_selection_3_null['ID']
df_clean_selection_3_null['ID'] = df_clean_selection_3_null['Year']
df_clean_selection_3_null['Year'] = df_clean_selection_3_null['Classification']
df_clean_selection_3_null['Classification'] = None

## Combine cleaned rows and export full output to CSV

In [ ]:
# drop df_clean rows where selection 3 is null
df_clean = df_clean[df_clean['Selection 3'].isnull() == False]

# add df_clean_selection_3_null to df_clean
df_clean = pd.concat([df_clean, df_clean_selection_3_null], ignore_index=True)

# Make df_clean column integers
df_clean['Year'] = df_clean['Year'].astype(int)

# sort df_clean by year
df_clean.sort_values(by=['Year'], inplace=True)

# convert to csv
df_clean.to_csv("full_output.csv")


In [10]:
# Select all 'Selection 1' rows as a list
selection_1_list = df_clean['Selection 1'].tolist()
# Select all 'Selection 2' rows
selection_2_list = df_clean['Selection 2'].tolist()
# Select all 'Selection 3' rows
selection_3_list = df_clean['Selection 3'].tolist()

# combine selection 1, 2, and 3 into one list
selection_list = selection_1_list + selection_2_list + selection_3_list

# convert list to dataframe
selection_df = pd.DataFrame(selection_list)
selection_df.columns = ['Selection']

# count the number of times each selection appears in selection_list
selection_count = selection_df['Selection'].value_counts()
selection_count

 Dance Celebration  (Smith)                                         776
 Moscow, 1941  (Balmages)                                           687
 Colliding Visions  (Balmages/ )                                    681
 Atlantis  (McGinty)                                                639
 Pinnacle  (Grice)                                                  592
                                                                   ... 
 El Capitan (Sousa/ Jack Bullock)                                     1
 Children's March "Over the Hills and Far Away" (Percy Grainger)      1
 The Diplomat (Sousa/Byrne)                                           1
 Queen City March ( W.H. Boorn)                                       1
 Fidelity March (K.L. King/Milford)                                   1
Name: Selection, Length: 31209, dtype: int64

# Get the full PML list to CSV

In [13]:
# import all of the pml csv files into a dataframe
files = os.path.join("/Users/macdkw/Class/uil_results/pml/", "*.csv")
files = glob.glob(files)
pml_df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
# conver pml_df to csv
pml_df.to_csv("pml_output.csv")